In [36]:
from bs4 import BeautifulSoup

import requests
# Send the mail
import smtplib
# email body
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
# system date and time manipulation
import datetime
import re

In [119]:
FUNDS = {
    'BlackRock Next Generation Technology Fund': 'https://endowus.com/investment-funds-list/blackrock-bgf-next-generation-technology-fund-LU2290526321',
    'BlackRock World Technology Fund': 'https://endowus.com/investment-funds-list/blackrock-bgf-world-technology-fund-LU1852331112',
    'JP Morgan China Fund': 'https://endowus.com/investment-funds-list/jp-morgan-china-a-share-opportunities-fund-LU1655091616',
    'Franklin Technology Fund': 'https://endowus.com/investment-funds-list/franklin-templeton-franklin-technology-fund-LU1803068979',
    'Franklin U.S. Fund': 'https://endowus.com/investment-funds-list/franklin-templeton-u.s.-opportunities-fund-LU0320765059',
    'Schroders Climate Change Fund': 'https://endowus.com/investment-funds-list/schroders-isf-global-climate-change-fund-LU0312595415',
    'Schroders Global Growth Fund': 'https://endowus.com/investment-funds-list/schroders-isf-global-sustainable-growth-fund-LU2289884723'
}

now = datetime.datetime.now()

In [136]:
content = '''
<head>
<style>
table {
  font-family: arial, sans-serif;
  border-collapse: collapse;
  width: 100%;
}

td, th {
  border: 1px solid #dddddd;
  text-align: left;
  padding: 8px;
}

tr:nth-child(even) {
  background-color: #dddddd;
}

a {
    color: #0472ff;
    text-decoration: none;
}
</style>
</head>
<body>

<h2>Investment Portfolio</h2>

<table>
  <tr>
    <th>Fund</th>
    <th>NAV</th>
    <th>Change</th>
    <th>Last Update</th>
  </tr>
'''

In [90]:
def get_fund_code(fund):
    return fund.split('-')[-1]

In [99]:
def get_day_change(soup):
    result = soup.find_all('span', class_='mod-ui-data-list__value')
    nav = result[0].text
    delta = result[1].text.split('/')[-1]
    delta = delta.strip()
    return nav, delta

In [114]:
def get_updated_date(soup):
    result = soup.find_all('div', class_='mod-disclaimer')
    result_list = result[0].text.split(' ')
    month = result_list[-3]
    day = result_list[-2]
    year = result_list[-1][:-1]
    return f'{day} {month}, {year}'

In [116]:
def get_fund_soup(fund_code):
    link = f'https://markets.ft.com/data/funds/tearsheet/summary?s={fund_code}:SGD'
    resp = requests.get(link)
    return BeautifulSoup(resp.content, 'html.parser')

In [134]:
def convert_to_html(fund_name, fund_detail, nav, delta, updated_date):
    return f'''
        <tr>
            <td>
                <a href={fund_detail} target="_blank">
                    {fund_name}
                </a>
            </td>
            <td>
                {nav}
            </td>
            <td>
                {delta}
            </td>
            <td>
                {updated_date}
            </td>
        </tr>
    '''

In [137]:
print("Scraping in progress")
for fund_name, fund_detail in FUNDS.items():
    print(f'Scraping {fund_name}')
    fund_code = get_fund_code(fund_detail)
    soup = get_fund_soup(fund_code)
    nav, delta = get_day_change(soup)
    updated_date = get_updated_date(soup)
    content += convert_to_html(fund_name, fund_detail, nav, delta, updated_date)
content.replace('\n', '')

'<head><style>table {  font-family: arial, sans-serif;  border-collapse: collapse;  width: 100%;}td, th {  border: 1px solid #dddddd;  text-align: left;  padding: 8px;}tr:nth-child(even) {  background-color: #dddddd;}a {    color: #0472ff;    text-decoration: none;}</style></head><body><h2>Investment Portfolio</h2><table>  <tr>    <th>Fund</th>    <th>NAV</th>    <th>Change</th>    <th>Last Update</th>  </tr>        <tr>            <td>                <a href=https://endowus.com/investment-funds-list/blackrock-bgf-next-generation-technology-fund-LU2290526321 target="_blank">                    BlackRock Next Generation Technology Fund                </a>            </td>            <td>                10.20            </td>            <td>                0.39%            </td>            <td>                05 Nov, 2021            </td>        </tr>            <tr>            <td>                <a href=https://endowus.com/investment-funds-list/blackrock-bgf-world-technology-fund-LU185

In [138]:
print('Composing Email')

SERVER = 'smtp.gmail.com' # "your smtp server"
PORT = 587 # your port number
FROM =  'bteocoding@gmail.com' # "your from email id"
TO = 'tteo43@gmail.com' # "your to email ids"  # can be a list
PASS = 'P@ssw0rd4523' # "your email id's password"

Composing Email


In [139]:
msg = MIMEMultipart()
msg['Subject'] = 'Daily Investment Portfolio Update' + ' ' + str(now.day) + '-' + str(now.month) + '-' + str(
    now.year)
msg['From'] = FROM
msg['To'] = TO
msg.attach(MIMEText(content, 'html'))

In [140]:
print('Initiating Server...')

server = smtplib.SMTP(SERVER, PORT)
#server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
server.set_debuglevel(1)
server.ehlo()
server.starttls()
#server.ehlo
server.login(FROM, PASS)
server.sendmail(FROM, TO, msg.as_string())

print('Email Sent...')

server.quit()

Initiating Server...


send: 'ehlo [192.168.18.12]\r\n'
reply: b'250-smtp.gmail.com at your service, [2404:e801:2001:d3d:41a9:cae5:cec1:a8c2]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'smtp.gmail.com at your service, [2404:e801:2001:d3d:41a9:cae5:cec1:a8c2]\nSIZE 35882577\n8BITMIME\nSTARTTLS\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8'
send: 'STARTTLS\r\n'
reply: b'220 2.0.0 Ready to start TLS\r\n'
reply: retcode (220); Msg: b'2.0.0 Ready to start TLS'
send: 'ehlo [192.168.18.12]\r\n'
reply: b'250-smtp.gmail.com at your service, [2404:e801:2001:d3d:41a9:cae5:cec1:a8c2]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-AUTH LOGIN PLAIN XOAUTH2 PLAIN-CLIENTTOKEN OAUTHBEARER XOAUTH\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
re

Email Sent...


reply: b'250 2.0.0 OK  1636362293 pi17sm10239309pjb.34 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.0.0 OK  1636362293 pi17sm10239309pjb.34 - gsmtp'
data: (250, b'2.0.0 OK  1636362293 pi17sm10239309pjb.34 - gsmtp')
send: 'quit\r\n'
reply: b'221 2.0.0 closing connection pi17sm10239309pjb.34 - gsmtp\r\n'
reply: retcode (221); Msg: b'2.0.0 closing connection pi17sm10239309pjb.34 - gsmtp'


(221, b'2.0.0 closing connection pi17sm10239309pjb.34 - gsmtp')